In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.dpi'] = 150
import glob, os

import scipy.stats as st
import timeit

# Regress viral severity on predictors

(severity) ~ (# of human targets) + (# of papers studying the virus) + (size of viral genomes) + etc.

In [2]:
# also rearranged so that all Homo sapiens is in the right column
virus_human = pd.read_csv("../Processed/virus_human.csv")
human_human = pd.read_csv("../Processed/human_important_for_virus.csv")

# don't need virus_virus, which is a dataframe of viral proteins interacting with each other

proteins = pd.read_csv("../Processed/enriched_GO_proteins.csv")

In [3]:
# check that all human proteins are in the second column
print(virus_human['Taxname B'].unique(), virus_human['Taxid B'].unique())

# human should not be in the first column
print('Homo sapiens' in virus_human['Taxname A'].values)
print(9606 in virus_human['Taxid A'].values)

['Homo sapiens'] [9606]
False
False


In [4]:
print(len(virus_human["Taxname A"].unique()), "viral species in the PPI dataset")

168 viral species in the PPI dataset


In [5]:
# save taxids to a text file to search with Taxonomizr in R (code below)
virus_human[["Taxid A"]].drop_duplicates().to_csv("viral_taxids.txt", index=False, sep="\t")

# Get taxonomic ranks using `Taxonomizr` in R

## Easiest way to collapse groups of viruses (i.e. different papillomaviruses will also have the same genus)

https://cran.r-project.org/web/packages/taxonomizr/vignettes/usage.html

```r
install.packages("taxonomizr")
library(taxonomizr)

prepareDatabase('accessionTaxa.sql')

df <- read.table("viral_taxids.txt", stringsAsFactors=FALSE, quote="", header=TRUE, sep="\t")

taxids <- df[["Taxid.A"]]

# get taxonomies for the taxids
taxa <- getTaxonomy(taxids,'accessionTaxa.sql')

# save taxa to a new dataframe
write.csv(taxa, "viral_taxonomies.csv")
```

In [6]:
viral_taxonomies = pd.read_csv("viral_taxonomies.csv")
viral_taxonomies.rename(columns={viral_taxonomies.columns[0]: 'Taxid A'}, inplace=True)

# combine the taxonomies with the original dataframe
virus_human = pd.merge(virus_human, viral_taxonomies, on='Taxid A')

# Get genome sizes of all the viruses

https://www.ncbi.nlm.nih.gov/genome/browse/#!/viruses/

Use taxonomizr gets the exact species name, rather than the strain name (which is what's associated with the taxids).

This species name should be easier to search than the taxid-associated taxname.

In [7]:
virus_genome_sizes = pd.read_csv("../Data/virus_genome_sizes.csv")

In [8]:
# ~1/2 as many species as taxids
len(virus_human.species.unique())

# make a dictionary to map between them
taxnames_species_dict = dict(zip(virus_human["Taxname A"], virus_human["species"]))

In [9]:
# this only has 62 of the 83 species in the dataframe. Compare families for the rest because the Organism Groups column contains them
found_viruses_df = virus_genome_sizes.loc[virus_genome_sizes["#Organism Name"].isin(virus_human['species'].unique())].drop_duplicates(["#Organism Name", "Size(Mb)"])

found_viruses = found_viruses_df["#Organism Name"].unique()

print(len(found_viruses), "viruses found in the database")

missing_viruses = list(set(virus_human['species'].unique()) - set(found_viruses))

print(len(missing_viruses), "viruses not a direct match")

62 viruses found in the database
21 viruses not a direct match


In [10]:
virus_genomes_df = pd.DataFrame(found_viruses_df.groupby("#Organism Name")["Size(Mb)"].mean()).reset_index()
virus_genomes_df.columns = ["Species", "Size(Mb)"]

add_viruses_to_df = []
genome_sizes = []

for virus in missing_viruses:
    
    res_df = virus_genome_sizes.loc[virus_genome_sizes["#Organism Name"].str.contains("|".join(virus.split(" ")), case=False)]
    if len(res_df) > 0:
        genome_sizes.append(res_df["Size(Mb)"].mean())
        add_viruses_to_df.append(virus)
    else:
        print(virus)
        
# add rubella virus manually because it's just 1 virus
add_viruses_to_df.append("Rubivirus rubellae")
genome_sizes.append(virus_genome_sizes.loc[virus_genome_sizes["#Organism Name"].str.contains("rubella", case=False)]["Size(Mb)"].mean())

virus_genomes_df = pd.concat([virus_genomes_df, pd.DataFrame({"Species": add_viruses_to_df, "Size(Mb)": genome_sizes})]).reset_index(drop=True)

assert len(set(virus_genomes_df["Species"]).intersection(virus_human["species"])) == len(virus_genomes_df)

Rubivirus rubellae


In [11]:
model_df = pd.DataFrame(taxnames_species_dict, index=[0]).T.reset_index()
model_df.columns = ["Taxname", "Species"]

# combine with the genome lengths
model_df = pd.merge(model_df, virus_genomes_df, on="Species", how="outer")

# number of proteins interacting with human proteins for a given viral species
protein_interactions = pd.DataFrame(virus_human.groupby("Taxname A")["Publication"].count()).reset_index()
protein_interactions.columns = ["Taxname", "num_interactions"]

# combine with the protein_interactions list
model_df = pd.merge(model_df, protein_interactions, on="Taxname", how="outer")

def num_unique_publications(df, taxname):
    
    df_single_tax = df.loc[df["Taxname A"] == taxname]    
    return len(np.unique(df_single_tax["Publication"]))

num_pubs = [num_unique_publications(virus_human, taxname) for taxname in model_df.Taxname.values]

model_df["Publications"] = num_pubs

# make sure that all numbers of publications are equal to or smaller than the number of protein interactions
assert sum(model_df["Publications"] <= model_df["num_interactions"]) == len(model_df)

# finally, save it to a dataframe for modeling later (need the outcome variable)
model_df.to_csv("severity_model_predictors.csv", index=False)

# Use fatality rate for viral severity

In [12]:
model_df

,Taxname,Species,Size(Mb),num_interactions,Publications
0,Betacoronavirus England 1,Middle East respiratory syndrome-related coron...,0.030119,590,20
1,Human betacoronavirus 2c EMC/2012,Middle East respiratory syndrome-related coron...,0.030119,6,4
2,Middle East respiratory syndrome-related coron...,Middle East respiratory syndrome-related coron...,0.030119,7,3
3,Severe acute respiratory syndrome-related coro...,Severe acute respiratory syndrome-related coro...,0.029274,1324,47
4,Severe acute respiratory syndrome coronavirus 2,Severe acute respiratory syndrome-related coro...,0.029274,3405,83
...,...,...,...,...,...
163,Human papillomavirus 4,Gammapapillomavirus 1,0.043874,2,1
164,Human papillomavirus type 96,Betapapillomavirus 5,0.007415,1,1
165,Murine minute virus strain MVM prototype,Rodent protoparvovirus 1,0.043923,1,1
166,Adeno-associated virus 2 Srivastava/1982,Adeno-associated dependoparvovirus A,0.048147,4,1
